# Selecting Logistic Regression Model with K-Fold CV

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler

In this notebook, we are interested in predicting biological characteristics ("phenotypes") from gene expression data of mice. Genes are the basic unit in the DNA and encode blueprints for proteins. When proteins are synthesized from a gene, the gene is said to "express". Micro-arrays are devices that measure the expression levels of large numbers of genes in parallel. By finding correlations between expression levels and phenotypes, scientists can identify possible genetic markers for biological characteristics.

The data in this notebook comes from mice. For each mouse, we have data about expression levels of 77 genes, and we also have some information about the mouse's biological characteristics. For example, some of the mice had Down Syndrome (trisomy), and this is indicated in the `Genotype` column in the data.

First, load in the data:

In [2]:
df = pd.read_csv('Data_Cortex_Nuclear.csv', index_col=0)

Get a quick view of the columns in this data by runnning the cell below. 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1080 entries, 309_1 to J3295_15
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   DYRK1A_N         1077 non-null   float64
 1   ITSN1_N          1077 non-null   float64
 2   BDNF_N           1077 non-null   float64
 3   NR1_N            1077 non-null   float64
 4   NR2A_N           1077 non-null   float64
 5   pAKT_N           1077 non-null   float64
 6   pBRAF_N          1077 non-null   float64
 7   pCAMKII_N        1077 non-null   float64
 8   pCREB_N          1077 non-null   float64
 9   pELK_N           1077 non-null   float64
 10  pERK_N           1077 non-null   float64
 11  pJNK_N           1077 non-null   float64
 12  PKCA_N           1077 non-null   float64
 13  pMEK_N           1077 non-null   float64
 14  pNR1_N           1077 non-null   float64
 15  pNR2A_N          1077 non-null   float64
 16  pNR2B_N          1077 non-null   float64
 17  pPKCAB_N   

We will use "has Down syndrome" as the target value to predict. We will prepare a variable `y` whose value is 0 or 1 depending on whether or not the mouse has Down syndrome.

In [4]:
txt, y = np.unique(df['Genotype'], return_inverse=True)

For the feature values we will use the gene expression levels:

In [5]:
x_names = ['DYRK1A_N', 'ITSN1_N', 'BDNF_N', 'NR1_N', 'NR2A_N', 'pAKT_N',
       'pBRAF_N', 'pCAMKII_N', 'pCREB_N', 'pELK_N', 'pERK_N', 'pJNK_N',
       'PKCA_N', 'pMEK_N', 'pNR1_N', 'pNR2A_N', 'pNR2B_N', 'pPKCAB_N',
       'pRSK_N', 'AKT_N', 'BRAF_N', 'CAMKII_N', 'CREB_N', 'ELK_N',
       'ERK_N', 'GSK3B_N', 'JNK_N', 'MEK_N', 'TRKA_N', 'RSK_N', 'APP_N',
       'Bcatenin_N', 'SOD1_N', 'MTOR_N', 'P38_N', 'pMTOR_N', 'DSCR1_N',
       'AMPKA_N', 'NR2B_N', 'pNUMB_N', 'RAPTOR_N', 'TIAM1_N', 'pP70S6_N',
       'NUMB_N', 'P70S6_N', 'pGSK3B_N', 'pPKCG_N', 'CDK5_N', 'S6_N',
       'ADARB1_N', 'AcetylH3K9_N', 'RRP1_N', 'BAX_N', 'ARC_N', 'ERBB4_N',
       'nNOS_N', 'Tau_N', 'GFAP_N', 'GluR3_N', 'GluR4_N', 'IL1B_N',
       'P3525_N', 'pCASP9_N', 'PSD95_N', 'SNCA_N', 'Ubiquitin_N',
       'pGSK3B_Tyr216_N', 'SHH_N', 'BAD_N', 'BCL2_N', 'pS6_N', 'pCFOS_N',
       'SYP_N', 'H3AcK18_N', 'EGR1_N', 'H3MeK4_N', 'CaNA_N']
X = df[x_names].values

Now, we will split the data into training and test sets using `sklearn`'s implementation of `train_test_split`. 

* Reserve 30% of the data for testing, and leave 70% for training.
* Shuffle the data, and use the random state specified in the PrairieLearn question page.

In [6]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
random_state = 7
Xtr,Xts,ytr,yts=train_test_split(X,y,random_state=random_state,test_size=0.3)

For some of the mice, some gene expression levels are missing. We will fill in the missing entries using  using the median values from the non-missing data *in the training set only*.

In [7]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
med_values = np.nanmedian(Xtr, axis=0)
Xtr_filled = np.nan_to_num(Xtr, nan=med_values)
Xts_filled = np.nan_to_num(Xts, nan=med_values)

Since many of the genes may be irrelevant for predicting whether or not a mouse has Down Syndrome, we can use L1 regularization to "de-select" some features.

In an `sklearn` `LogisticRegression`, the hyperparameter `C` controls the "strength" of the regularization term in the objective function. `C` is the **inverse** of the regularization strength; a greater value of `C` means the model is *less* regularized.

We will evaluate models for the following values of `C`:

In [8]:
C_test = np.logspace(-1,3,10)

In [18]:
C_test.shape

(10,)

In the following cells, we are going to set up a K-fold CV to select a value of `C`.  First, we will set up an array to hold the results of each model in each fold. (Note that our K-fold CV will use 3 folds.)

In [9]:
nfold = 3
acc_val = np.zeros((len(C_test), nfold))

Now, create a KFold object using the `sklearn` implementation. Use 3 folds (and don't shuffle the data inside the K-Fold CV). 

When using a regularized model, we should standardize the data (remove the mean and scale to unit variance) first. Use the `sklearn` implementation of a `StandardScaler` to standardize data in each fold. Save the results in `Xtr_std` and `Xvl_std`.

Use this to evalute an `sklearn` `LogisticRegression` regression model for each of the `C` values in `C_test`, and save the validation accuracy inside `acc_val`. In the `LogisticRegression`, 

* specify `solver = 'liblinear'`
* specify `penalty = 'l1'`
* specify `random_state` again (using the value from the PrairieLearn question page)

and leave other hyperparameters and settings at their default values, except for `C`.

In [10]:
acc_val.shape

(10, 3)

In [ ]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)

# Set random_state as per the question's requirement
random_state = 7

nfold = 3 

# Initialize KFold (without shuffling data)
kf = KFold(n_splits=nfold, shuffle=False)

# KFold cross-validation
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(Xtr_filled)):
    # Split data into training and validation sets
    X_train, X_val = Xtr_filled[train_idx],Xtr_filled[val_idx]
    y_train, y_val = ytr[train_idx], ytr[val_idx]
    
    # Standardize the training and validation data using StandardScaler
    scaler = StandardScaler()
    
    # Fit scaler on the training data and transform both training and validation data
    X_train_std = scaler.fit_transform(X_train)
    X_val_std = scaler.transform(X_val)
    
    # For each value of C in C_test, train and evaluate the Logistic Regression model
    for c_idx, C in enumerate(C_test):
        clf = LogisticRegression(solver='liblinear', penalty='l1', C=C, random_state=random_state)
        clf.fit(X_train_std, y_train)  # Train the model
        y_val_pred = clf.predict(X_val_std)  # Make predictions on the validation set
        acc_val[c_idx, fold_idx] = accuracy_score(y_val, y_val_pred)  # Store the accuracy


Next, compute the mean validation accuracy for each of the models, and identify the value of `C` for which the validation accuracy is maximized.

In [ ]:
Xtr_filled.shape

In [15]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
# Calculate the mean accuracy across all folds for each value of C
acc_mean = np.mean(acc_val, axis=1)

# Find the value of C that gives the highest mean accuracy
C_best = C_test[np.argmax(acc_mean)]

Using the `C` value you identified in the previous step (and other logistic regression parameters as specified earlier), we will fit a logistic regression model on the entire training set.

In [17]:
acc_mean.shape

(10,)

First, fit a `StandardScaler` on the entire training set, and use it to standardize both the training and test sets. Save the results in `Xtr_std` and `Xts_std`.

In [19]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
scaler=StandardScaler()

Xtr_std = scaler.fit_transform(Xtr_filled)
Xts_std = scaler.transform(Xts_filled)

Then, fit a logistic regression model with the "best" C, and get its test set prediction in `y_hat`.

In [23]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
model=LogisticRegression(C=C_best,solver='liblinear', penalty='l1',  random_state=random_state)

model.fit(Xtr_std,ytr)
y_hat = model.predict(Xts_std)

---

_Attribution_: Based on a question by Professor Sundeep Rangan.